##  `ColumnTransformer` (열 변환기)

`ColumnTransformer`는 사이킷런(Scikit-learn)의 전처리 도구 중 가장 강력하고 유연한 도구로, \*\*서로 다른 데이터 열(Columns)\*\*에 **서로 다른 전처리 작업**을 독립적으로 적용한 다음, 그 결과를 하나의 데이터셋으로 결합하는 역할을 합니다.

이는 파이프라인(Pipeline)과 결합하여 머신러닝 모델 학습 전에 데이터 전처리 과정을 체계적이고 오류 없이 수행할 수 있게 해줍니다.

-----

### 1\. `ColumnTransformer`의 역할과 필요성

#### 1.1 역할

  * **다중 전처리 동시 수행:** 수치형 데이터에는 스케일링을, 범주형 데이터에는 인코딩을, 특정 열에는 누락값 처리를 동시에 적용할 수 있습니다.
  * **결과 병합:** 각 변환기(Transformer)의 출력 결과를 열(Column) 단위로 병합하여 다음 단계(예: 모델 학습)로 전달합니다.

#### 1.2 필요성

대부분의 실제 데이터셋은 수치형, 범주형, 텍스트 등 다양한 타입의 특성(Feature)을 포함하고 있으며, 각 타입은 고유한 전처리 방식(StandardScaler, OneHotEncoder 등)을 필요로 합니다. `ColumnTransformer`가 없다면 각 전처리 단계를 수동으로 관리하고 결과를 합쳐야 하는 복잡한 과정을 거쳐야 합니다.

-----

### 2\. 기본 구조 및 주요 매개변수

`ColumnTransformer`는 `transformers`라는 핵심 매개변수를 통해 정의됩니다.

#### 2.1 `transformers` 매개변수 (핵심)

`transformers`는 리스트 형태로, 각 요소는 다음 세 가지를 담는 튜플로 구성됩니다.

$$(\text{변환기 이름 (Name)}, \text{변환기 객체 (Transformer)}, \text{적용할 열 (Columns)})$$

| 구성 요소 | 설명 | 예시 |
| :--- | :--- | :--- |
| **Name** | 변환 단계를 식별하는 문자열 이름 (사용자 정의). | `'num_scaler'`, `'cat_encoder'` |
| **Transformer** | 해당 열에 적용할 사이킷런 변환기 객체. (`StandardScaler()`, `OneHotEncoder()`, `SimpleImputer()` 등) 또는 `Pipeline` 객체. | `StandardScaler()`, `Pipeline([...])` |
| **Columns** | 변환을 적용할 데이터프레임의 열(특성). 열 이름의 리스트, 인덱스, 슬라이스, 또는 `make_column_selector`를 사용할 수 있습니다. | `['Age', 'Fare']`, `[0, 2]`, `make_column_selector(dtype_include=np.number)` |

#### 2.2 `remainder` 매개변수 (나머지 열 처리)

`transformers`에 명시되지 않은 나머지 열을 어떻게 처리할지 지정합니다.

| 값 | 동작 방식 |
| :--- | :--- |
| **`'drop'`** (기본값) | 명시되지 않은 모든 열을 **삭제**합니다. |
| **`'passthrough'`** | 명시되지 않은 모든 열을 **변환 없이 그대로** 최종 결과에 포함합니다. (가장 일반적인 설정) |
| **변환기 객체** | 나머지 모든 열에 특정 변환기를 적용합니다. |

-----

### 3\. 실용적인 구성 예시

수치형 특성에는 스케일링을, 범주형 특성에는 인코딩을 적용하고, 누락값을 처리하는 일반적인 예시입니다.

```python
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import numpy as np

# 1. 특성 목록 정의 (실제 데이터에 맞게 수정)
numerical_features = ['Age', 'Fare']
categorical_features = ['Embarked']

# 2. 전처리 파이프라인 구성

# 수치형 전처리 (누락값 중앙값 대체 -> 스케일링)
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 범주형 전처리 (누락값 최빈값 대체 -> 원-핫 인코딩)
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# 3. ColumnTransformer 정의
preprocessor = ColumnTransformer(
    transformers=[
        # 수치형 특성에는 numerical_pipeline 적용
        ('num_pipe', numerical_pipeline, numerical_features),
        # 범주형 특성에는 categorical_pipeline 적용
        ('cat_pipe', categorical_pipeline, categorical_features)
    ],
    remainder='drop' # 정의되지 않은 모든 열은 삭제
    # remainder='passthrough' 를 사용하면 정의되지 않은 열을 그대로 유지합니다.
)

# 4. 전체 모델 Pipeline에 결합하여 사용
# model_cpipe = Pipeline([ ('pre', preprocessor), ('logi', LogisticRegression()) ])
# model_cpipe.fit(X_train, y_train)
```

이 구조는 데이터 전처리 과정을 명확하게 분리하고, 전체 과정을 `Pipeline`에 통합하여 \*\*데이터 누수(Data Leakage)\*\*를 방지하고 코드를 간결하게 유지할 수 있도록 해줍니다.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# 1. 예시 데이터 생성 (Pandas DataFrame 사용)
data = {
    'Age': [30.0, np.nan, 22.0, 35.0, np.nan, 40.0, 22.0],
    'Fare': [7.25, 71.28, 7.92, 53.1, 8.05, 30.0, 7.0],
    'Embarked': ['S', 'C', 'S', 'S', 'Q', np.nan, 'S'] # np.nan은 object 타입에서 문자열 'nan'이 아닌 실제 결측치
}
X_data = pd.DataFrame(data)

# 2. 특성(열) 목록 정의
# 누락값을 채워야 하는 수치형 특성과 범주형 특성을 분리
num_impute_feature = ['Age']
num_scale_feature = ['Fare']
cat_impute_feature = ['Embarked']

# 3. 전처리 파이프라인(ColumnTransformer) 구성
column_preprocessor = ColumnTransformer(
    transformers=[
        # =========================================================
        # 1. 수치형 - 중앙값(Median) 대체 후 스케일링
        # Pipeline을 사용하여 Imputer와 Scaler를 순차적으로 적용합니다.
        ('num_impute_pipe',
         Pipeline([
             ('imputer', SimpleImputer(strategy='median')),
             ('scaler', StandardScaler())
         ]),
         num_impute_feature
        ),
        # =========================================================
        # 2. 수치형 - 스케일링만 적용
        ('num_scale',
         StandardScaler(),
         num_scale_feature
        ),
        # =========================================================
        # 3. 범주형 - 최빈값(Most Frequent) 대체 후 One-Hot 인코딩
        ('cat_impute_pipe',
         Pipeline([
             ('imputer', SimpleImputer(strategy='most_frequent')),
             ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)) # sparse_output=False로 배열로 출력 설정
         ]),
         cat_impute_feature
        )
    ],
    remainder='drop' # 목록에 없는 다른 모든 열은 삭제
)

# 4. ColumnTransformer 적용
X_processed = column_preprocessor.fit_transform(X_data)

# 5. 결과 확인
print("--- 원본 데이터 ---")
print(X_data)
print("\n--- ColumnTransformer 적용 후 결과 (NumPy Array) ---")
# 결과는 (샘플 수, 변환된 특성 수) 형태의 NumPy 배열입니다.
print(X_processed)
print(f"\n변환 후 데이터 형태: {X_processed.shape}")